# Workshop_04
## Immagine di esempio:

<img src= './img01.jpg'>

Per la costruzione del tetto è stato deciso di utilizzare un'intersezione fra 2 modelli presenti nell'immagine sopra:
* Mansard Roof
* Intersecting Roof

Sono stati definiti due tipi di codice per la funzione principale:
* tipo 1 : la funzione principale prende in input un hpc value di un tetto.
* tipo 2 : la funzione principale prende in input vertici e celle.

## Tipo 1 ------------ 
### Codice:

Il Codice è composto da più funzioni:
* una funzione ausiliaria planarity_check(verts) che controlla la planarità di n vertici.
* una funzione ausiliaria faces_planarity_check(verts,cells) che dati in input vertici e celle verifica la planarità dei vertici per ogni cella.
* una funzione ausiliaria remove_2DCells(cells) che banalmente rimuove le celle di due dimensioni utilizzate per creare gli assi di sostegno interni alla struttura.
* una funzione principale che dato un hpc value di un tetto restituisce la struttura completa di travi di sostegno e copertura del tetto.

In [9]:
from larlib import *
from numpy import *

"""ggpl_mansard_intersecting_roof_builder è una funzione che dato in input un hpc value di un tetto
ne verifica la planarità, dopodiche ne estrae vertici e celle per costruirne una struttura in 3D dotata
di travi di supporto e copertura
"""
def ggpl_mansard_intersecting_roof_builder1(hpc):
    
#FUNZIONI AUSILIARIE ------------------------------------------------------------------------#

    """in an n-dimensional space (n>=3) a set of k points are cooplanar if and only if 
    the matrix of their relative differences is of rank 2 or less
    """
    def planarity_check(verts):
        #meno di 4 punti = coplanare
        if(len(verts)>3):
            lastPoint = verts[-1]
            checkMatrix = verts[0:-1]
            for i in range(len(checkMatrix)):
                #ad ogni punto rimasto sottraggo l'ultimo punto
                checkMatrix[i] = np.subtract(checkMatrix[i],lastPoint)
            if(linalg.matrix_rank(matrix(checkMatrix))>2): #controllo rank
                return False
        return True

    def faces_planarity_check(verts, cells):
        for cell in cells:
            vertsToCheck = []
            for i in cell:
                 vertsToCheck += [verts[i-1]] 
            if(not planarity_check(vertsToCheck)):
                return False
        return True
    
    def remove_2DCell(cells):
        newCells = []
        for cell in cells:
            if len(cell)>2:
                newCells += [cell]
        return newCells

#FINE FUNZIONI AUSILIARIE ----------------------------------------------------------------#
    
    ukInfo = UKPOL(SKEL_2(hpc))
    if(not faces_planarity_check(ukInfo[0],ukInfo[1])): #ckeck planarity tramite f. ausiliaria
        return None
    
    beams = STRUCT([OFFSET([0.2,0.2,0.2])(SKEL_1(hpc))]) #struttura delle travi
    
    verts = list([round(x,3),round(y,3), round(z,3)] for [x,y,z] in ukInfo[0])
    
    cells = ukInfo[1]
    newCells = remove_2DCell(cells) #rimuovo cell 2D tramite f. ausiliaria
    
    faces = STRUCT([OFFSET([0.2,0.2,0.2]),(MKPOL([verts, newCells, None]))])
    
    return STRUCT([T(3)(10),beams,T([3])([0.2]),COLOR(RED)(faces)])


### Esempi:

In [10]:
#TETTO DI PARTENZA---------------------------------------------------------------------------#
verts = [[7.,5.,5.],[7.,-5.,5.],[-7.,5.,5.],[-7.,-5.,5.], #rect interno
          [14.,10.,0],[14.,-10.,0],[-14.,10.,0],[-14.,-10.,0], #rect esterno
          [0,0,8.], #punta centrale
          [0,5.,5.],[0,10.,5.],[0,16.,5.],#punta prolungamento
          [-7.,10.,0],[7.,10.,0],[-7.,16.,0],[7.,16.,0], #base prolungamento
          [7.,5.,0],[7.,-5.,0],[-7.,5.,0],[-7.,-5.,0] #punti con z=0 per travi basse
        ]  

cells = [[1,2,5,6],[1,3,5,7],[3,4,7,8],[2,4,6,8], #connessioni rect interna-esterna
          [1,2,9],[1,3,9],[3,4,9],[2,4,9], #connessioni rect interna-puntacentrale
          [10,11,13],[10,11,14],[11,12,15,13],[11,12,16,14],[12,15,16], #connessioni prolungamento
          [17,18],[17,19],[18,20],[19,20],[1,17],[2,18],[4,20],[3,19],[5,17],[6,18],[8,20],[7,19]
        ] 

startRoof = (MKPOL([verts,cells,None])) #tetto di partenza

VIEW(ggpl_mansard_intersecting_roof_builder1(startRoof))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000004EAF450> >

<img src = './ris01.png'>

<img src = './ris02.png'>

In [11]:
#TETTO DI PARTENZA---------------------------------------------------------------------------#
verts = [[5.,5.,5.],[5.,-5.,5.],[-5.,5.,5.],[-5.,-5.,5.], #rect interno
          [7.,10.,0],[7.,-10.,0],[-7.,10.,0],[-7.,-10.,0], #rect esterno
          [0,0,15.], #punta centrale
          [0,5.,5.],[0,10.,5.],[0,10.,5.],#punta prolungamento
          [-5.,10.,0],[5.,10.,0],[-5.,16.,0],[5.,16.,0], #base prolungamento
          [5.,5.,0],[5.,-5.,0],[-5.,5.,0],[-5.,-5.,0] #punti con z=0 per travi basse
        ]  

cells = [[1,2,5,6],[1,3,5,7],[3,4,7,8],[2,4,6,8], #connessioni rect interna-esterna
          [1,2,9],[1,3,9],[3,4,9],[2,4,9], #connessioni rect interna-puntacentrale
          [10,11,13],[10,11,14],[11,12,15,13],[11,12,16,14],[12,15,16], #connessioni prolungamento
          [17,18],[17,19],[18,20],[19,20],[1,17],[2,18],[4,20],[3,19],[5,17],[6,18],[8,20],[7,19]
        ] 

startRoof = (MKPOL([verts,cells,None])) #tetto di partenza

VIEW(ggpl_mansard_intersecting_roof_builder1(startRoof))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000004EAF360> >

<img src = './res03.png'>

<img src = './res04.png'>

## Tipo 2 ------------ 
### Codice:

Il Codice è composto da più funzioni:
* una funzione ausiliaria planarity_check(verts) che controlla la planarità di n vertici.
* una funzione ausiliaria faces_planarity_check(verts,cells) che dati in input vertici e celle verifica la planarità dei vertici per ogni cella.
* una funzione ausiliaria remove_2DCells(cells) che banalmente rimuove le celle di due dimensioni utilizzate per creare gli assi di sostegno interni alla struttura.
* una funzione principale che dati in input vertici e celle della struttura di un tetto restituisce la struttura completa di travi di sostegno e copertura del tetto.

In [15]:
from larlib import *
from numpy import *

def ggpl_mansard_intersecting_roof_builder2(verts, cells):
    
#FUNZIONI AUSILIARIE ------------------------------------------------------------------------#

    """in an n-dimensional space (n>=3) a set of k points are cooplanar if and only if 
    the matrix of their relative differences is of rank 2 or less
    """
    def planarity_check(verts):
        #meno di 4 punti = coplanare
        if(len(verts)>3):
            lastPoint = verts[-1]
            checkMatrix = verts[0:-1]
            for i in range(len(checkMatrix)):
                #ad ogni punto rimasto sottraggo l'ultimo punto
                checkMatrix[i] = np.subtract(checkMatrix[i],lastPoint)
            if(linalg.matrix_rank(matrix(checkMatrix))>2): #controllo rank
                return False
        return True

    def faces_planarity_check(verts, cells):
        for cell in cells:
            vertsToCheck = []
            for i in cell:
                 vertsToCheck += [verts[i-1]] 
            if(not planarity_check(vertsToCheck)):
                return False
        return True
    
    def remove_2DCell(cells):
        newCells = []
        for cell in cells:
            if len(cell)>2:
                newCells += [cell]
        return newCells

#FINE FUNZIONI AUSILIARIE ----------------------------------------------------------------#
    
    startRoof = (MKPOL([verts,cells,None])) #tetto di partenza
    
    beams = STRUCT([OFFSET([0.2,0.2,0.2])(SKEL_1(startRoof))]) #struttura delle travi

    ukInfo = UKPOL(SKEL_2(startRoof))
    if(not faces_planarity_check(ukInfo[0],ukInfo[1])): #ckeck planarity tramite f. ausiliaria
        return None

    newCells = remove_2DCell(cells) #rimuovo cell 2D tramite f. ausiliaria
    
    faces = STRUCT([OFFSET([0.2,0.2,0.2]),(MKPOL([verts, newCells, None]))])

    return STRUCT([T(3)(10),beams,T([3])([0.2]),COLOR(RED)(faces)])

### Esempi:

In [17]:
#TETTO DI PARTENZA---------------------------------------------------------------------------#
verts = [[7.,5.,5.],[7.,-5.,5.],[-7.,5.,5.],[-7.,-5.,5.], #rect interno
          [14.,10.,0],[14.,-10.,0],[-14.,10.,0],[-14.,-10.,0], #rect esterno
          [0,0,8.], #punta centrale
          [0,5.,5.],[0,10.,5.],[0,16.,5.],#punta prolungamento
          [-7.,10.,0],[7.,10.,0],[-7.,16.,0],[7.,16.,0], #base prolungamento
          [7.,5.,0],[7.,-5.,0],[-7.,5.,0],[-7.,-5.,0] #punti con z=0 per travi basse
        ]  

cells = [[1,2,5,6],[1,3,5,7],[3,4,7,8],[2,4,6,8], #connessioni rect interna-esterna
          [1,2,9],[1,3,9],[3,4,9],[2,4,9], #connessioni rect interna-puntacentrale
          [10,11,13],[10,11,14],[11,12,15,13],[11,12,16,14],[12,15,16], #connessioni prolungamento
          [17,18],[17,19],[18,20],[19,20],[1,17],[2,18],[4,20],[3,19],[5,17],[6,18],[8,20],[7,19]
        ] 

VIEW(ggpl_mansard_intersecting_roof_builder2(verts,cells))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000004E2C120> >

<img src = './ris01.png'>

<img src = './ris02.png'>

In [11]:
#TETTO DI PARTENZA---------------------------------------------------------------------------#
verts = [[5.,5.,5.],[5.,-5.,5.],[-5.,5.,5.],[-5.,-5.,5.], #rect interno
          [7.,10.,0],[7.,-10.,0],[-7.,10.,0],[-7.,-10.,0], #rect esterno
          [0,0,15.], #punta centrale
          [0,5.,5.],[0,10.,5.],[0,10.,5.],#punta prolungamento
          [-5.,10.,0],[5.,10.,0],[-5.,16.,0],[5.,16.,0], #base prolungamento
          [5.,5.,0],[5.,-5.,0],[-5.,5.,0],[-5.,-5.,0] #punti con z=0 per travi basse
        ]  

cells = [[1,2,5,6],[1,3,5,7],[3,4,7,8],[2,4,6,8], #connessioni rect interna-esterna
          [1,2,9],[1,3,9],[3,4,9],[2,4,9], #connessioni rect interna-puntacentrale
          [10,11,13],[10,11,14],[11,12,15,13],[11,12,16,14],[12,15,16], #connessioni prolungamento
          [17,18],[17,19],[18,20],[19,20],[1,17],[2,18],[4,20],[3,19],[5,17],[6,18],[8,20],[7,19]
        ] 

VIEW(ggpl_mansard_intersecting_roof_builder2(verts,cells))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000004EAF360> >

<img src = './res03.png'>

<img src = './res04.png'>